# COMP30760 Assignment 1 - Task 1

In this assignment I will be collecting data from the Genius API. Genius is world's biggest collection of song lyrics and musical knowledge. Recently 2 very famouse artists, Drake and Kanye West Reseased their albums called Certified Lover Boy (CLB) and Donda respectively. There was a huge of debate all over the world regarding which album is better. Although this a subjective, I decided to use the Genius API to find an answer to this. 

Genius API only provides current information about the songs so I decided to fetch the data regarding each song on an hourly basis. I did this so I can get sufficient data to analyse and find out the popularity trend of each album.  

In [1]:
import json, requests, urllib
from pathlib import Path
from datetime import datetime
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
!pip install lyricsgenius
import lyricsgenius
%matplotlib inline

In [294]:
# API Key (replace with your own API key)
api_key = "qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku"
# Prefix for API URLs
api_prefix = "https://api.genius.com"
# python library to get the lyrics of songs
LyricsGenius = lyricsgenius.Genius(api_key)
#name of the artists 
artist_names = ["Drake", "Kanye West"]
#songs in CLB by Drake
CLB_Songs = ["Champagne poetry", "Papi's home" , "Girls Want Girls", "In The Bible", "Love All", "Fair Trade", "Way 2 Sexy", "TSU", "N 2 Deep", "Pipe Down", "No Friends In The Industry", "Knife Talk", "7am On Bridle Path", "Race My Mind", "Fountains", "Get Along Better", "You Only Live Twice", "IMY2", "Fucking Fans", "The Remorse"] 
#songs in Donda by Kanye West
Donda_Songs = ["Donda Chant", "Jail", "God Breathed", "Off the Grid", "Praise God", "Jonah", "Ok Ok", "Junya", "Believe What I Say", "24", "Remote Control", "Moon", "Heaven and Hell", "Donda", "Keep My Spririt Alive", "Jesus Lord", "New Again", "Tell The Vision", "Lord I Need You", "Pure Souls", "Come To Life", "No Child Left Behind", "Jail pt 2", "Ok Ok pt 2", "Junya pt 2", "Jesus Lord pt 2"]


created directories for the raw JSON data

In [311]:
dir_raw = Path("raw")
dir_raw_lyrics = Path("raw/lyrics")
dir_raw.mkdir(parents=True, exist_ok=True)

## Data Collection
made functions to easily fetch the data for Genius API

In [271]:
def fetch(endpoint, params={}):
    # construct the url
    url = api_prefix
    if not endpoint.startswith("/"):
        url += "/"
    url += endpoint
    (api_prefix, endpoint)
    params["access_token"] = api_key
    url += "?" + urllib.parse.urlencode(params)
    print("Fetching %s" % url)
    # fetch the page
    response = requests.get(url)
    jdata = response.text
    return json.loads(jdata)

Genius API has a Search function which outputs all the top results of the information inputted. I used this function to fetch each song and then save them in a folder named "raw" as .JSON files.

In [273]:
def fetchSongs(album, artist, metadata):
    for song in album:
        CLB_data = fetch("/search", {"q":song})
        for result in CLB_data['response']['hits']:
            # is this the result we are looking for?
            if result['result']['primary_artist']['name'] == artist:
                metadata[song] = result 
                date_suffix = datetime.now().strftime("%Y%m%d-%H00")
                fname = "%s-%s-%s.json" % (song, artist, date_suffix)
                out_path = dir_raw / fname
                print("Writing data to %s" % out_path)
                fout = open(out_path, "w")
                json.dump(result, fout, indent=4, sort_keys=True)
                fout.close()
                break
       
    return(metadata)

to fetch lyrics I used a python library called LyricsGenius. 

In [321]:
def fetchLyrics(album, artist):
    for song in album:
        LyricsGenius.remove_section_headers = True
        lyrics = LyricsGenius.search_song(artist, song)
        fname = "%s.txt" % (song)
        out_path = dir_raw_lyrics / fname
        print("Writing data to %s" % out_path)
        fout = open(out_path, "wb")
        pickle.dump(lyrics.lyrics, fout)
        fout.close()
        
        print(lyrics)

I ran the code below multiple times to get enough statistical data about the songs to analyse. 

In [340]:
CLB_metadata = {}
Donda_metadata = {}

fetchSongs(CLB_Songs, artist_names[0], CLB_metadata)
fetchSongs(Donda_Songs, artist_names[1], Donda_metadata)

print(CLB_metadata)
print(Donda_metadata)


Fetching https://api.genius.com/search?q=Champagne+poetry&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/Champagne poetry-Drake-20211105-2200.json
Fetching https://api.genius.com/search?q=Papi%27s+home&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/Papi's home-Drake-20211105-2200.json
Fetching https://api.genius.com/search?q=Girls+Want+Girls&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/Girls Want Girls-Drake-20211105-2200.json
Fetching https://api.genius.com/search?q=In+The+Bible&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/In The Bible-Drake-20211105-2200.json
Fetching https://api.genius.com/search?q=Love+All&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/Love All-Drake-20211105-2200.json
Fetching https://api.genius.com/search?q=F

Writing data to raw/Ok Ok pt 2-Kanye West-20211105-2200.json
Fetching https://api.genius.com/search?q=Junya+pt+2&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/Junya pt 2-Kanye West-20211105-2200.json
Fetching https://api.genius.com/search?q=Jesus+Lord+pt+2&access_token=qCd-olJOGrh5Vi_W-K9iRXibNQxoZIwlvzaLlJtIDQfaL4GoD6k7CBeBOOdzP7ku
Writing data to raw/Jesus Lord pt 2-Kanye West-20211105-2200.json
{'Champagne poetry': {'highlights': [], 'index': 'song', 'type': 'song', 'result': {'annotation_count': 15, 'api_path': '/songs/6859296', 'artist_names': 'Drake', 'full_title': 'Champagne Poetry by\xa0Drake', 'header_image_thumbnail_url': 'https://images.genius.com/fa02d8bc4c7ee74b5a1408c2be032fea.300x300x1.png', 'header_image_url': 'https://images.genius.com/fa02d8bc4c7ee74b5a1408c2be032fea.1000x1000x1.png', 'id': 6859296, 'lyrics_owner_id': 5748418, 'lyrics_state': 'complete', 'path': '/Drake-champagne-poetry-lyrics', 'pyongs_count': 24, '

In [322]:
fetchLyrics(CLB_Songs,artist_names[0])

Searching for "Drake" by Champagne poetry...
Done.
Writing data to raw/lyrics/Champagne poetry.txt
"Champagne Poetry" by Drake:
    I love you, I love you, I love you
    Until I, until I
    I love you, I love you, I love you
    Until I find ...
Searching for "Drake" by Papi's home...
Done.
Writing data to raw/lyrics/Papi's home.txt
"Papi’s Home" by Drake:
    I know that I hurt you
    Yeah
    I was a child trying to be a man
    To all my sons worldwide
    I walked out o...
Searching for "Drake" by Girls Want Girls...
Done.
Writing data to raw/lyrics/Girls Want Girls.txt
"Girls Want Girls" by Drake:
    Woah, woah
    Woah, woah, woah
    Ayy
    
    Starin' at your dress 'cause it's see-through
    Yeah, talkin' all the...
Searching for "Drake" by In The Bible...
Done.
Writing data to raw/lyrics/In The Bible.txt
"In The Bible" by Drake:
    Okay, okay, okay
    Countin' up my bodies, damn, the thing just start to pile
    Judge me how you judge me...
Searching for "Drake" by Lo

In [315]:
fetchLyrics(Donda_Songs,artist_names[1])

Searching for "Kanye West" by Donda Chant...
Done.
Writing data to raw/lyrics/Donda Chant.txt
"Donda Chant" by Kanye West:
    [Interlude: Syleena Johnson]
    Donda, Donda, Donda, Donda, Donda, Donda
    Donda, Donda, Donda, Donda, Do...
Searching for "Kanye West" by Jail...
Done.
Writing data to raw/lyrics/Jail.txt
"Jail" by Kanye West:
    [Intro: Kanye West]
    Take what you want
    Take everything
    Take what you want
    Take what you want
    
    [Verse...
Searching for "Kanye West" by God Breathed...
Done.
Writing data to raw/lyrics/God Breathed.txt
"God Breathed" by Kanye West:
    [Chorus: Kanye West]
    I know God breathed on this
    I know God breathed on this
    I know God breathed on ...
Searching for "Kanye West" by Off the Grid...
Done.
Writing data to raw/lyrics/Off the Grid.txt
"Off The Grid" by Kanye West:
    [Intro: Kanye West & Playboi Carti]
    What? Yeah
    Boy (What?)
    
    [Chorus: Kanye West]
    We off the grid, gr...
Searching for "Kanye West" 

the table below shows some of the data about the songs. I used this to chech if i fetched the correct data.

In [338]:
metadata_rows = []
for song in CLB_Songs:
    row = {"Song": song, "ID": CLB_metadata[song]['result']['id']}
    row["hot"] = CLB_metadata[song]['result']['stats']['hot']
    metadata_rows.append(row)
pd.DataFrame(metadata_rows).set_index("Song")

,ID,hot
Song,,
Champagne poetry,6859296,False
Papi's home,7165569,False
Girls Want Girls,7165581,False
In The Bible,7165576,False
Love All,5367025,False
Fair Trade,6833653,False
Way 2 Sexy,7139486,False
TSU,5312186,False
N 2 Deep,7163722,False
